# **Data augmentation: POS-driven method**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pickle
import numpy as np

from nltk.tag.stanford import StanfordPOSTagger
from nltk.tokenize import word_tokenize

from nltk.corpus import wordnet

from lxml import html
import requests

# Install word tokenizer:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Install French POS-tagger:
!wget 'https://nlp.stanford.edu/software/stanford-tagger-4.2.0.zip'
!unzip stanford-tagger-4.2.0.zip

# Install Huggingface libraries:
!pip install transformers

from transformers import pipeline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


--2023-05-23 19:11:13--  https://nlp.stanford.edu/software/stanford-tagger-4.2.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-tagger-4.2.0.zip [following]
--2023-05-23 19:11:14--  https://downloads.cs.stanford.edu/nlp/software/stanford-tagger-4.2.0.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78034596 (74M) [application/zip]
Saving to: ‘stanford-tagger-4.2.0.zip’

stanford-tagger-4.2 100%[===================>]  74.42M  5.07MB/s    in 11s     

2023-05-23 19:11:24 (7.05 MB/s) - ‘stanford-tagger-4.2.0.zip’ saved [78034596/78034596]

Archive:  stanford-tagger-4.2.0.zip
   creating:

## **Load the data**

Classification task:

In [4]:
task = '3'

Load train set:

In [5]:
f_in = open("drive/MyDrive/train_set_"+task+"_orig.pkl","rb")

data_train = pickle.load(f_in)
 
f_in.close()

Extract positive examples:

In [6]:
data_train_positive = [data_train[i][0] for i in range(len(data_train)) if data_train[i][2]]

In [7]:
data_train_positive[0:5]

['Article 1 : Occupations ou utilisations du sol interdites\n \n1) Dans l’ensemble de la zone sont interdits :\n \nLes terrains de camping ou de caravanage permanents visés à l’article L.443-1 et L.444-1 du \ncode de l’urbanisme.',
 'Article 1 : Occupations ou utilisations du sol interdites\n \n1) Dans l’ensemble de la zone sont interdits :\n \nLes habitations légères de loisirs.',
 'Article 1 : Occupations ou utilisations du sol interdites\n \n1) Dans l’ensemble de la zone sont interdits :\n \nLes constructions destinées à l’habitation ne dépendant pas d’une exploitation agricole autres \nque celles visées à l’article 2 paragraphe 1).',
 'Article 1 : Occupations ou utilisations du sol interdites\n \n1) Dans l’ensemble de la zone sont interdits :\n \nLes constructions destinées à l’hébergement hôtelier autres que celles visées à l’article 2 \nparagraphe 1).',
 'Article 1 : Occupations ou utilisations du sol interdites\n \n1) Dans l’ensemble de la zone sont interdits :\n \nLes construct

## **Perform augmentation**

### **POS-driven method (adj+adv): replace all adjectives and adverbs in each segment using masked word prediction**

Experiment (augmentation) name:

In [8]:
experiment = '1'

How many times repeat augmentation:

In [9]:
k = 1

Define POS-tagger:

In [10]:
st = StanfordPOSTagger('/content/stanford-postagger-full-2020-11-17/models/french-ud.tagger',
                       '/content/stanford-postagger-full-2020-11-17/stanford-postagger-4.2.0.jar',
                       encoding='utf-8')

Perform POS-tagging, then mask all adjectives and adverbs in each phrase:

In [11]:
classified_segments = []
masked_segments = []
for i in range(len(data_train_positive)):
  tokenized_text = word_tokenize(data_train_positive[i], language='french')
  classified_text = st.tag(tokenized_text)
  masked_text = ""
  for word,tag in classified_text:
    if tag != 'ADJ' and tag != 'ADV':
      if word == "’" or word == ".":
        masked_text = masked_text[:-1] + word
      else:
        masked_text += word + " "
    else:
      masked_text += '<mask> '
  classified_segments.append(classified_text)
  masked_segments.append(masked_text)
  if i % 10 == 0:
    print("Process",i,"segment")

Process 0 segment
Process 10 segment
Process 20 segment
Process 30 segment
Process 40 segment
Process 50 segment
Process 60 segment
Process 70 segment
Process 80 segment
Process 90 segment
Process 100 segment
Process 110 segment


In [12]:
masked_segments[0:5]

['Article 1 : Occupations ou utilisations du sol interdites 1 ) Dans l’<mask> de la zone sont interdits : Les terrains de camping ou de caravanage <mask> visés à l’article L.443-1 et L.444-1 du code de l’urbanisme.',
 'Article 1 : Occupations ou utilisations du sol interdites 1 ) Dans l’<mask> de la zone sont interdits : Les habitations <mask> de loisirs.',
 'Article 1 : Occupations ou utilisations du sol interdites 1 ) Dans l’<mask> de la zone sont interdits : Les constructions destinées à l’habitation <mask> dépendant <mask> d’une exploitation <mask> <mask> que celles visées à l’article 2 paragraphe 1 ).',
 'Article 1 : Occupations ou utilisations du sol interdites 1 ) Dans l’<mask> de la zone sont interdits : Les constructions destinées à l’hébergement <mask> <mask> que celles visées à l’article 2 paragraphe 1 ).',
 'Article 1 : Occupations ou utilisations du sol interdites 1 ) Dans l’<mask> de la zone sont interdits : Les constructions destinées <mask> bureaux , au commerce et acti

Predict masked words in the resulting sentences:

In [13]:
model_name = "camembert-base" 

camembert_unmasker = pipeline("fill-mask", model=model_name, tokenizer=model_name) # define the model

def generateCombinations(masked_text):
  # predict masked words:
  predicted_words = camembert_unmasker(masked_text)

  # extract predicted words:
  generated_words = []
  if masked_text.count("<mask>") == 1:
    generated_candidates = []
    for word in predicted_words[:k]:
      generated_candidates.append(word['token_str'])
    generated_words.append(generated_candidates)
  else:
    for word_candidates in predicted_words:
      generated_candidates = []
      for word in word_candidates[:k]:
        generated_candidates.append(word['token_str'])
      generated_words.append(generated_candidates)

  return generated_words

def maskedPhrase2NewText(generated_combinations,combination_id,classified_phrase):
  predicted_text = ""
  word_id = 0
  for word,tag in classified_phrase:
    if tag != 'ADJ' and tag != 'ADV':
      if word == "’" or word == ".":
        predicted_text = predicted_text[:-1] + word
      else:
        predicted_text += word + " "
    else:
      predicted_text += generated_combinations[word_id][combination_id] + " "
      word_id += 1

  return predicted_text

In [14]:
new_training_set = []
count_without_mask = 0
count_already_exist = 0

for i in range(len(masked_segments)):
  nb_combinations = masked_segments[i].count("<mask>")
  if nb_combinations > 0:
    generated_combinations = generateCombinations(masked_segments[i])
    for j in range(k):
      new_phrase = maskedPhrase2NewText(generated_combinations,j,classified_segments[i])
      if new_phrase.strip() != data_train_positive[i].replace("\n","").strip():
        new_training_set.append(new_phrase)
      else:
        count_already_exist += 1
  else:
    count_without_mask += 1
  if i % 10 == 0:
    print("Process",i,"segment")

Process 0 segment
Process 10 segment
Process 20 segment
Process 30 segment
Process 40 segment
Process 50 segment
Process 60 segment
Process 70 segment
Process 80 segment
Process 90 segment
Process 100 segment
Process 110 segment


Generated phrases:

In [15]:
new_training_set[0:5]

['Article 1 : Occupations ou utilisations du sol interdites 1 ) Dans l’ensemble de la zone sont interdits : Les terrains de camping ou de caravanage sont visés à l’article L.443-1 et L.444-1 du code de l’urbanisme.',
 'Article 1 : Occupations ou utilisations du sol interdites 1 ) Dans l’ensemble de la zone sont interdits : Les habitations principales de loisirs.',
 'Article 1 : Occupations ou utilisations du sol interdites 1 ) Dans l’ensemble de la zone sont interdits : Les constructions destinées à l’habitation ne dépendant pas d’une exploitation agricole autre que celles visées à l’article 2 paragraphe 1 ).',
 'Article 1 : Occupations ou utilisations du sol interdites 1 ) Dans l’ensemble de la zone sont interdits : Les constructions destinées à l’hébergement ( autres que celles visées à l’article 2 paragraphe 1 ).',
 'Article 1 : Occupations ou utilisations du sol interdites 1 ) Dans l’ensemble de la zone sont interdits : Les constructions destinées aux bureaux , au commerce et activ

Create new segments:

In [16]:
data_new = [(i,-1,True) for i in new_training_set]

data_augmented = data_train + data_new

Some stats:

In [17]:
len(data_augmented)

573

In [18]:
print("Positive examples:", len([i for i in range(len(data_augmented)) if data_augmented[i][2]]))
print("Negative examples:", len([i for i in range(len(data_augmented)) if not data_augmented[i][2]]))

Positive examples: 221
Negative examples: 352


Save results:

In [19]:
f_out = open("drive/MyDrive/train_set_"+task+"_augm-"+experiment+".pkl","wb")

pickle.dump(data_augmented,f_out)

f_out.close()

### **POS-driven method (nouns): replace all nouns in each segment using masked word prediction**

Experiment (augmentation) name:

In [20]:
experiment = '2'

How many times repeat augmentation:

In [21]:
k = 2

Perform POS-tagging, then mask all adjectives and adverbs in each phrase:

In [22]:
classified_segments = []
masked_segments = []
for i in range(len(data_train_positive)):
  tokenized_text = word_tokenize(data_train_positive[i], language='french')
  classified_text = st.tag(tokenized_text)
  masked_text = ""
  for word,tag in classified_text:
    if tag != 'NOUN':
      if word == "’" or word == ".":
        masked_text = masked_text[:-1] + word
      else:
        masked_text += word + " "
    else:
      masked_text += '<mask> '
  classified_segments.append(classified_text)
  masked_segments.append(masked_text)
  if i % 10 == 0:
    print("Process",i,"segment")

Process 0 segment
Process 10 segment
Process 20 segment
Process 30 segment
Process 40 segment
Process 50 segment
Process 60 segment
Process 70 segment
Process 80 segment
Process 90 segment
Process 100 segment
Process 110 segment


In [23]:
masked_segments[0:5]

['<mask> 1 : <mask> ou <mask> du <mask> interdites 1 ) Dans l <mask> ensemble de la <mask> sont interdits : Les <mask> de <mask> ou de <mask> permanents visés à l’<mask> L.443-1 et L.444-1 du <mask> de l’<mask>.',
 '<mask> 1 : <mask> ou <mask> du <mask> interdites 1 ) Dans l <mask> ensemble de la <mask> sont interdits : Les <mask> légères de <mask>.',
 '<mask> 1 : <mask> ou <mask> du <mask> interdites 1 ) Dans l <mask> ensemble de la <mask> sont interdits : Les <mask> destinées à l’<mask> ne dépendant pas d’une <mask> agricole autres que celles <mask> à l’<mask> 2 <mask> 1 ).',
 '<mask> 1 : <mask> ou <mask> du <mask> interdites 1 ) Dans l <mask> ensemble de la <mask> sont interdits : Les <mask> destinées à l’<mask> hôtelier autres que celles <mask> à l’<mask> 2 <mask> 1 ).',
 '<mask> 1 : <mask> ou <mask> du <mask> interdites 1 ) Dans l <mask> ensemble de la <mask> sont interdits : Les <mask> destinées aux <mask> , au <mask> et <mask> de <mask> , à l’<mask> , à l’<mask> autres que celle

Predict masked words in the resulting sentences:

In [24]:
def generateCombinations(masked_text):
  # predict masked words:
  predicted_words = camembert_unmasker(masked_text)

  # extract predicted words:
  generated_words = []
  if masked_text.count("<mask>") == 1:
    generated_candidates = []
    for word in predicted_words[:k]:
      generated_candidates.append(word['token_str'])
    generated_words.append(generated_candidates)
  else:
    for word_candidates in predicted_words:
      generated_candidates = []
      for word in word_candidates[:k]:
        generated_candidates.append(word['token_str'])
      generated_words.append(generated_candidates)

  return generated_words

def maskedPhrase2NewTextNOUN(generated_combinations,combination_id,classified_phrase):
  predicted_text = ""
  word_id = 0
  for word,tag in classified_phrase:
    if tag != 'NOUN':
      if word == "’" or word == ".":
        predicted_text = predicted_text[:-1] + word
      else:
        predicted_text += word + " "
    else:
      predicted_text += generated_combinations[word_id][combination_id] + " "
      word_id += 1

  return predicted_text

**Note:** All possible combinations of predicted words (their Cartesian product) are not taken into account. Instead, k different phrases are generated regardless the number of masks in a phrase

In [25]:
new_training_set = []
count_without_mask = 0
count_already_exist = 0

for i in range(len(masked_segments)):
  nb_combinations = masked_segments[i].count("<mask>")
  if nb_combinations > 0:
    generated_combinations = generateCombinations(masked_segments[i])
    for j in range(k):
      new_phrase = maskedPhrase2NewTextNOUN(generated_combinations,j,classified_segments[i])
      if new_phrase.strip() != data_train_positive[i].replace("\n","").strip():
        new_training_set.append(new_phrase)
      else:
        count_already_exist += 1
  else:
    count_without_mask += 1
  if i % 10 == 0:
    print("Process",i,"segment")

Process 0 segment
Process 10 segment
Process 20 segment
Process 30 segment
Process 40 segment
Process 50 segment
Process 60 segment
Process 70 segment
Process 80 segment
Process 90 segment
Process 100 segment
Process 110 segment


Generated phrases:

In [26]:
new_training_set[0:5]

['1 1 : les ou de du des interdites 1 ) Dans l l ensemble de la de sont interdits : Les de de le ou de les permanents visés à l’article L.443-1 et L.444-1 du de de l’».',
 'de 1 : de ou du du site interdites 1 ) Dans l ‘ ensemble de la qui sont interdits : Les et de la ou de de permanents visés à l’Article L.443-1 et L.444-1 du et de l’de.',
 'Page 1 : Les ou parties du corps interdites 1 ) Dans l L ensemble de la France sont interdits : Les armes légères de et.',
 'Chapitre 1 : les ou activités du sexe interdites 1 ) Dans l l ensemble de la société sont interdits : Les drogues légères de :.',
 '1 1 : l ou de du sont interdites 1 ) Dans l l ensemble de la et sont interdits : Les activités destinées à l’et ne dépendant pas d’une exploitation agricole autres que celles de à l’( 2 ( 1 ).']

Create new segments:

In [27]:
data_new = [(i,-1,True) for i in new_training_set]

data_augmented = data_train + data_new

Some stats:

In [28]:
len(data_augmented)

706

In [29]:
print("Positive examples:", len([i for i in range(len(data_augmented)) if data_augmented[i][2]]))
print("Negative examples:", len([i for i in range(len(data_augmented)) if not data_augmented[i][2]]))

Positive examples: 354
Negative examples: 352


Save results:

In [30]:
f_out = open("drive/MyDrive/train_set_"+task+"_augm-"+experiment+".pkl","wb")

pickle.dump(data_augmented,f_out)

f_out.close()

### **POS-driven method (verbs): replace all verbs in each segment using masked word prediction**

Experiment (augmentation) name:

In [31]:
experiment = '3'

How many times repeat augmentation:

In [32]:
k = 3

Perform POS-tagging, then mask all adjectives and adverbs in each phrase:

In [33]:
classified_segments = []
masked_segments = []
for i in range(len(data_train_positive)):
  tokenized_text = word_tokenize(data_train_positive[i], language='french')
  classified_text = st.tag(tokenized_text)
  masked_text = ""
  for word,tag in classified_text:
    if tag != 'VERB':
      if word == "’" or word == ".":
        masked_text = masked_text[:-1] + word
      else:
        masked_text += word + " "
    else:
      masked_text += '<mask> '
  classified_segments.append(classified_text)
  masked_segments.append(masked_text)
  if i % 10 == 0:
    print("Process",i,"segment")

Process 0 segment
Process 10 segment
Process 20 segment
Process 30 segment
Process 40 segment
Process 50 segment
Process 60 segment
Process 70 segment
Process 80 segment
Process 90 segment
Process 100 segment
Process 110 segment


In [34]:
masked_segments[0:5]

['Article 1 : Occupations ou utilisations du sol <mask> 1 ) Dans l’ensemble de la zone sont <mask> : Les terrains de camping ou de caravanage permanents <mask> à l’article L.443-1 et L.444-1 du code de l’urbanisme.',
 'Article 1 : Occupations ou utilisations du sol <mask> 1 ) Dans l’ensemble de la zone sont <mask> : Les habitations légères de loisirs.',
 'Article 1 : Occupations ou utilisations du sol <mask> 1 ) Dans l’ensemble de la zone sont <mask> : Les constructions <mask> à l’habitation ne <mask> pas d’une exploitation agricole autres que celles visées à l’article 2 paragraphe 1 ).',
 'Article 1 : Occupations ou utilisations du sol <mask> 1 ) Dans l’ensemble de la zone sont <mask> : Les constructions <mask> à l’hébergement hôtelier autres que celles visées à l’article 2 paragraphe 1 ).',
 'Article 1 : Occupations ou utilisations du sol <mask> 1 ) Dans l’ensemble de la zone sont <mask> : Les constructions <mask> aux bureaux , au commerce et activités de service , à l’artisanat , à 

Predict masked words in the resulting sentences:

In [35]:
def generateCombinations(masked_text):
  # predict masked words:
  predicted_words = camembert_unmasker(masked_text)

  # extract predicted words:
  generated_words = []
  if masked_text.count("<mask>") == 1:
    generated_candidates = []
    for word in predicted_words[:k]:
      generated_candidates.append(word['token_str'])
    generated_words.append(generated_candidates)
  else:
    for word_candidates in predicted_words:
      generated_candidates = []
      for word in word_candidates[:k]:
        generated_candidates.append(word['token_str'])
      generated_words.append(generated_candidates)

  return generated_words

def maskedPhrase2NewTextVERB(generated_combinations,combination_id,classified_phrase):
  predicted_text = ""
  word_id = 0
  for word,tag in classified_phrase:
    if tag != 'VERB':
      if word == "’" or word == ".":
        predicted_text = predicted_text[:-1] + word
      else:
        predicted_text += word + " "
    else:
      predicted_text += generated_combinations[word_id][combination_id] + " "
      word_id += 1

  return predicted_text

**Note:** All possible combinations of predicted words (their Cartesian product) are not taken into account. Instead, k different phrases are generated regardless the number of masks in a phrase

In [36]:
new_training_set = []
count_without_mask = 0
count_already_exist = 0

for i in range(len(masked_segments)):
  nb_combinations = masked_segments[i].count("<mask>")
  if nb_combinations > 0:
    generated_combinations = generateCombinations(masked_segments[i])
    for j in range(k):
      new_phrase = maskedPhrase2NewTextVERB(generated_combinations,j,classified_segments[i])
      if new_phrase.strip() != data_train_positive[i].replace("\n","").strip():
        new_training_set.append(new_phrase)
      else:
        count_already_exist += 1
  else:
    count_without_mask += 1
  if i % 10 == 0:
    print("Process",i,"segment")

Process 0 segment
Process 10 segment
Process 20 segment
Process 30 segment
Process 40 segment
Process 50 segment
Process 60 segment
Process 70 segment
Process 80 segment
Process 90 segment
Process 100 segment
Process 110 segment


Generated phrases:

In [37]:
new_training_set[0:5]

['Article 1 : Occupations ou utilisations du sol ( 1 ) Dans l’ensemble de la zone sont concernés : Les terrains de camping ou de caravanage permanents mentionnés à l’article L.443-1 et L.444-1 du code de l’urbanisme.',
 'Article 1 : Occupations ou utilisations du sol Article 1 ) Dans l’ensemble de la zone sont autorisés : Les terrains de camping ou de caravanage permanents définis à l’article L.443-1 et L.444-1 du code de l’urbanisme.',
 'Article 1 : Occupations ou utilisations du sol : 1 ) Dans l’ensemble de la zone sont situés : Les terrains de camping ou de caravanage permanents visés à l’article L.443-1 et L.444-1 du code de l’urbanisme.',
 'Article 1 : Occupations ou utilisations du sol ( 1 ) Dans l’ensemble de la zone sont concernées : Les habitations légères de loisirs.',
 'Article 1 : Occupations ou utilisations du sol Article 1 ) Dans l’ensemble de la zone sont autorisées : Les habitations légères de loisirs.']

Create new segments:

In [38]:
data_new = [(i,-1,True) for i in new_training_set]

data_augmented = data_train + data_new

Some stats:

In [39]:
len(data_augmented)

818

In [40]:
print("Positive examples:", len([i for i in range(len(data_augmented)) if data_augmented[i][2]]))
print("Negative examples:", len([i for i in range(len(data_augmented)) if not data_augmented[i][2]]))

Positive examples: 466
Negative examples: 352


Save results:

In [41]:
f_out = open("drive/MyDrive/train_set_"+task+"_augm-"+experiment+".pkl","wb")

pickle.dump(data_augmented,f_out)

f_out.close()